In [108]:
import pandas as pd
import requests
import os
import subprocess
import json
import time
import re
import whisper
import sounddevice as sd
import numpy as np
import wave
import threading
from pydub import AudioSegment
import gc
import signal

In [96]:
model = whisper.load_model("medium.en")

100%|█████████████████████████████████████| 1.42G/1.42G [02:50<00:00, 8.99MiB/s]


In [2]:
# Construct the context from cached community reporting files
context = ""
for idx, e in enumerate(os.listdir(os.path.join("ragtest", "cache", "community_reporting"))):
    with open(os.path.join("ragtest", "cache", "community_reporting", e), 'r') as file:
        file_content = file.read()
        json_data = json.loads(file_content)
    title = eval(json_data["result"])["title"]
    summary = eval(json_data["result"])["summary"]
    context += f"Cluster {idx+1} theme: "
    context += f"Title: {title} \n"
    context += f"Summary: {summary} \n\n"
print(context)

Cluster 1 theme: Title: Biggie, Jay-Z, and Bad Boy Records 
Summary: The community centers around the influential figures of Biggie (The Notorious B.I.G.) and Jay-Z, with significant ties to Bad Boy Records and its recording studio, Daddy's House. The relationships between these entities highlight a network of collaboration, mutual respect, and shared history in the music industry. 

Cluster 2 theme: Title: Jay-Z and the Song 'FALLIN'' 
Summary: The community revolves around the song 'FALLIN'' by Jay-Z, featuring Bilal. The song references several classic crime films such as 'Scarface', 'Casino', 'Godfather', and 'Goodfellas' to illustrate the inevitable downfall of a life of crime. The relationships between these entities highlight the cultural and thematic significance of the song. 

Cluster 3 theme: Title: Jay-Z and His Musical Network 
Summary: This community revolves around Jay-Z, also known as Hov, and includes key figures and entities such as Biggie, Kanye West, and various nota

In [3]:
# Dialogue feed examples
dialogue_feed_1 = "I am a college graduate who likes software and would like to build profitable cool stuff. What is your advice?"
dialogue_feed_2 = "What is the meaning of life?"
dialogue_feed_3 = "How do you feel about shrimp?"

In [39]:
s = time.time()
g_counter = 0

while(g_counter<3):
    method = "local"  # or "global"
    query = f"""
    SHORT TERM MEMORY CONTEXT:
    {context}

    
    
    PROMPT:
    You are Jay Z, the influential rapper-mogul engaged in a conversation. The short term memory context provided has a high level overview of your experiences,
    and the short term dialogue context provided has examples of how you speak in conversation.
    Using a mixture of the context provided above and your long term memory, which consists of far more granular facts and experiences,
    please answer the following question as Jay Z would in person:
    {dialogue_feed_3}
    """
    
    # Run the bash script with the dynamic method and query
    subprocess.run(["./run_query.sh", method, query])
    
    # Wait until the file is created or updated
    output_file = 'output.txt'
    break_bool = False
    while not os.path.exists(output_file) or os.path.getmtime(output_file) < time.time() - 1:
        time.sleep(1)
        e = time.time()
        if (e-s) > 30:
            break_bool = True
            break
    if break_bool: 
        print(f"Breaking for iteration {g_counter+1}")
        g_counter+=1
        s = time.time()
        break
    
    # Read and display the content of the output file
    with open(output_file, 'r') as file:
        output = file.read()
    
    # Extract the relevant part of the output and clean it
    match = re.search(r'Local Search Response:(.*)', output, re.DOTALL)
    
    if match:
        success_text = match.group(1).strip()
        cleaned_text = re.sub(r'\[.*?\]', '', success_text)
        print(cleaned_text)
        g_counter = 9999999999
    else:
        print(f"No match found for iteration {g_counter+1}")

# Note: Real-time generation of this would be ideal, but this is a simple implementation for now.


NameError: name 'context' is not defined

In [117]:
import subprocess
import time

# Function to start recording without printing output
def start_recording():
    subprocess.run(["bash", "start_recording.sh"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Function to stop recording without printing output
def stop_recording():
    subprocess.run(["bash", "stop_recording.sh"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [123]:
recorder.start_recording()

Recording started, saving to output.wav


In [124]:
recorder.stop_recording()

Recording stopped, saved to output.wav


In [89]:
def transcribe_audio_segment(file_path, start_minute = 0, end_minute= 9999):
    # Load the audio file with pydub
    audio = AudioSegment.from_file(file_path, format="mp3")

    # Calculate start and end times in milliseconds
    start_time = start_minute * 60 * 1000
    end_time = end_minute * 60 * 1000

    # Extract the desired segment
    audio_segment = audio[start_time:end_time]

    # Export the segment as a temporary WAV file
    temp_wav = "temp_segment.wav"
    audio_segment.export(temp_wav, format="wav")

    # Transcribe the extracted segment using Whisper
    result = model.transcribe(temp_wav)
    print("Transcription:", result["text"])

In [ ]:
transcribe_audio_segment("FoodTox Lecture03-SD.mp3", start_minute=0, end_minute=5)

In [ ]:
#this worked.
# transcribe_audio_segment("FoodTox Lecture03-SD.mp3", start_minute=3, end_minute=5)


In [118]:
transcribe_audio_segment("output.wav")

/opt/homebrew/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:  dealership after photos of a truck he had traded in began circulating on social media because it was being used by ISIS to fight on the front lines in Syria and still had his company name
